In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
from dash.dependencies import Input, Output, State

import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from pymongo import MongoClient


import re 
import base64
# defining AC = Aaron Ciminelli. AC will be added to comments added by me 
# AC change animal_shelter and AnimalShelter 
from acAnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################


username = "aacuser" # AC Username required by instructions 
password = "password" # AC password set during setup
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


# AC Adding Grazioso Salvare's Logo



# AC Adding Grazioso Salvare's Logo
image_filename = 'GSL.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div(style={'background-color': '#f2f2f2'}, children=[
    # AC: Add a link to the SNHU website with the university logo image
    # AC: Define a link that opens in a new tab and displays the SNHU logo
    
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                            height=80, width=81))],
        href='https:/www.snhu.edu', target="_blank",
        style={
            'position': 'absolute',
            'top': '0',
            'right': '0',
        }),
    # AC: Add a title for the dashboard and the animal shelter name
    # AC: Define two titles for the dashboard, which are displayed in the center of the page

    html.Div([
        html.Div(html.Center(html.B(html.H1('Aaron Ciminelli CS-340 Dashboard',
                                            style={
                                                'color': 'black',
                                                'background-color': '#FF6347',
                                                'text-align': 'center',
                                            }))),
                 style={'display': 'inline-block',
                        'padding-left': '20px',
                        'padding-right': '20px'}),
        html.Div(html.Center(html.B(html.H1('Animal Shelter',
                                            style={'color': 'black',
                                                   'background-color': '#87CEEB',
                                                   'text-align': 'top-right',
                                                  }))),
                 style={'display': 'inline-block',
                        'padding-left': '20px',
                        'padding-right': '20px'}),
    ],
        style={'text-align': 'center'}),

    html.Hr(),

    # Add buttons for filtering data by rescue type, a dropdown menu for filtering data by breed,
    # and a data table for displaying the filtered data
    # AC: Define buttons for filtering data by rescue type, a dropdown menu for filtering data by breed,
    # and a data table for displaying the filtered data, displayed in a horizontal layout
    html.Br(), 
    html.Button('Retrieve All', id='button-id', style={'margin-left': '10px'}),
 html.Div([
        dcc.RadioItems(
            id='filter-radio',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'Water rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness rescue'},
                {'label': 'Disaster/Tracking Rescue', 'value': 'Disaster/Tracking rescue'},
            ],
            value='all',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        ),
        dcc.Dropdown(
            id='dropdown-id',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'Water rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                {'label': 'Disaster/Tracking Rescue', 'value': 'Disaster/Tracking escue'},
            ],
            value=None,
            clearable=True,
            style={'width': '200px', 'margin-left': '10px', 'margin-right': '10px'}
        )
    ], style={'display': 'flex', 'flex-wrap': 'wrap', 'justify-content': 'center'}),


    # AC: Define a table for displaying the animal shelter data
    # AC: Define columns for the table based on the animal shelter data, with some columns excluded
    # AC: Set various styles for the table, including the header, cells, and alternating row colors
    # AC: Add options for filtering, sorting, and selecting rows in the table
    # AC: Set the initial data for the table to be all rows in the animal shelter data
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            
            {"id": i, "deletable": False,
             "selectable": True} for i in df.columns if i not in ["gender", "age"]
        ],
        data=df.to_dict('records'),
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_cell={
            'backgroundColor': 'rgb(248, 248, 248)',
            'textAlign': 'center',
            'whiteSpace': 'normal',
            'textOverflow': 'ellipsis',
            'overflow': 'hidden',
            'boxShadow': '0px 0px 4px rgba(0, 0, 0, 0.25)'
        },
        style_data_conditional=[
            {'if': {'row_index': 'odd'},
             'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        filter_action='native',
        sort_action='native',
        page_action='native',
        row_selectable="single",
        selected_rows=[],
        page_size=20,
        editable= True
    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content':'center'},
             children=[

        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it




@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-radio', 'value')],
)
def update_data(rescue_type):
    if rescue_type == 'all':
        query = {}
    else:
        query = {'rescue_type': {'$regex': rescue_type, '$options': 'i'}}

    df = pd.DataFrame.from_records(shelter.read(query))
    return df.to_dict('records')

#change to derived_viewport_data if other behavior is wanted


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dff = pd.DataFrame.from_dict(viewData)
    top_breeds = dff.groupby('breed').size().reset_index(name='count').sort_values('count', ascending=False).head(10)

    return [
    # AC: Define a graph that displays the count of each animal breed in the filtered data
    # AC: Use plotly express to create a bar chart of the top 10 breeds by count
        dcc.Graph(
            figure = px.bar(top_breeds,
                            x='breed', y='count',
                            title='Counts by Breed',
                            labels={'breed': 'Breed', 'count': 'Count'}),
            style={'box-shadow': '0px 0px 20px 5px #FF6347'}
        )
    ]


#call back for slecting a row and then plotting the geomarker
@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(virtualRows):
    #austin Texas is [30.75, -97.48]
    
    #create the views
    if not virtualRows: #build a default view if there are no selected lines
        markerArray = (30.75,-97.48) #default marker at Austin Animal Shelter
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
        
    else: #build the contextual views based on the selection
        dff = pd.DataFrame(df.iloc[virtualRows]) #convert the datatable to a dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) #strip out the lat
        coordLong = float(dff['location_long'].to_string().split()[1]) #strip out the long
        markerArray = (coordLat, coordLong) #build the array based on selection
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    #return the map with a child marker
    #marker is set to the values found in markerArray
    #map centers/moves to view the new marker instead of holding a fixed center
     # AC: Define a map that displays a marker at the location of the selected animal in the table
    # AC: Define a map with a tile layer, and a child marker at the location of the selected animal
    return [dl.Map(style={'width': '700px', 'height': '450px', 'box-shadow': '0px 0px 20px 5px #FF6347'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]

app

Your Username and Password is: aacuser password
Connection Was Successful
